In [ ]:
import tempfile, shutil, os

In [ ]:
if os.environ.get('GOOGLE_APPLICATION_CREDENTIALS', None):
    !gcloud auth activate-service-account --key-file $GOOGLE_APPLICATION_CREDENTIALS

In [ ]:
snapshot_path = os.environ.get('PROMETHEUS_SNAPSHOT_PATH', '/data/snapshots')
bucket_name = os.environ.get('GCS_BUCKET', None)
prom_host = os.environ.get('PROMETHEUS_HOST', 'localhost')
prom_port = int(os.environ.get('PROMETHEUS_PORT', 9090))
slack_url = os.environ.get('SLACK_API_URL', None)
prom_namespace =  os.environ.get('PROMETHEUS_NAMESPACE', 'default')
prom_container =  os.environ.get('PROMETHEUS_CONTAINER', ' prometheus-server')

if bucket_name is None:
    raise Exception("GCS_BUCKET must be specified!")
    
if slack_url is None:
    raise Exception("SLACK_API_URL must be specified!")

In [ ]:
import http.client,datetime,os,requests, json

In [ ]:
def create_snapshot(https, host, port):
    conn = None
    try:
        if https:
            conn = http.client.HTTPSConnection(host, port=port)
        else:
            conn = http.client.HTTPConnection(host, port=port)
        conn.request( "POST", "/api/v1/admin/tsdb/snapshot")
        response = conn.getresponse()
    except Exception as e:
        print(e)
        return (False, str(e))
    if response.status != 200:
        msg = f"Request failed on snapshot creation! Status code: {response.status}({ response.reason})  {response.read()}"
        print(msg)
        return (False, msg)
    result = json.load(response)
    if conn != None:
        conn.close()
    return (True, result['data']['name'])

In [ ]:
(success, snapshot_name) = create_snapshot(False, prom_host, prom_port)

if success:
    tempdir = tempfile.mkdtemp()
    try:
        pods =  !kubectl get pod -l app=prometheus,component=server -o custom-columns=:metadata.name
        prom_pod = pods[1]
        if snapshot_path == '' or snapshot_name == '':
            raise Exception(f'snapshot_path and snapshot_name cannot be empty! (snapshot_name: {snapshot_name}, snapshot_path: {snapshot_path})')
        !kubectl cp -n {prom_namespace} -c {prom_container} {prom_pod}:{snapshot_path}/{snapshot_name} {tempdir}/{snapshot_name}
        !gsutil -m cp -r {tempdir}/{snapshot_name} gs://{bucket_name}/{snapshot_name}
    finally:
        if snapshot_path != '' and snapshot_name != '': 
          !kubectl exec -n {prom_namespace} -c {prom_container} {prom_pod} -- rm -rf {snapshot_path}/{snapshot_name}
        shutil.rmtree(tempdir)
else:
    now = datetime.datetime.now()
    message = str(now) + "\n" + snapshot_name
    data = {'text': message}
    requests.post(slack_url, json=data)